In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [ ]:
xe136cross=pd.read_csv('../crosssections/Xe136_N_XSec.csv',engine='python',header=None,delim_whitespace=True)
xe131cross=pd.read_csv('../crosssections/Xe131_N_XSec.csv',engine='python',header=None,delim_whitespace=True)
He3cross=pd.read_csv('../crosssections/He3_N_XSec.csv',engine='python',header=None,delim_whitespace=True)

In [ ]:
plt.figure(figsize=(7,7))

plt.plot(xe131cross[0]*1e-6,xe131cross[1],label="$^{131}$Xe",color='xkcd:teal',lw=3)
plt.plot(xe136cross[0]*1e-6,xe136cross[1],label="$^{136}$Xe",color='xkcd:plum',lw=3)
plt.plot(He3cross[0]*1e-6,He3cross[1],label="$^{3}$He",color='xkcd:orange',lw=3)

plt.semilogx()
plt.semilogy()
plt.yticks(fontsize=18)  
plt.xticks(fontsize=18) 
plt.ylim(5e-5,5000)  
plt.xlim(25e-9,20)
plt.ylabel("n Capture Cross Section (barns)",fontsize=20)
plt.xlabel('Incident energy (MeV)',fontsize=20)
plt.tight_layout()
plt.legend(fontsize=18,loc='upper right')
plt.show()

In [ ]:
acti_file = '../Xe137Count_sim87799000muons.h5'
# acti_file = '../outputs/Muons.next.h5'
muonnrgsproducingXe137 = pd.read_hdf(acti_file)


In [ ]:
# Number of simulated Muons
numberofmuons=87799000 

# Binning Settings
low_lim  = 1     # GeV
high_lim = 3000  # GeV
stp_size = 100
bins        = np.linspace(low_lim, high_lim, 30) # 30 bins from 1 - 3k GeV
bin_centers = bins[:-1] + np.diff(bins) / 2      # Shift values to bin centers

# Create a flat distribution from 1 - 3k GeV
nrgs = np.random.uniform(low_lim, high_lim, numberofmuons) 

# Make histogram from distribution with 1 - 3k GeV binning
binned_sim_muons, _ = np.histogram(nrgs, bins=bins)
plt.plot(binned_sim_muons)

In [ ]:
# Plot the muon energies from 1 - 3k GeV
binned_Xe137data    = plt.hist(muonnrgsproducingXe137['Xemunrg']*10**-3, bins)


In [ ]:
# Load in the datframe again
xe137_df = pd.read_hdf(acti_file)

# Add an extra column converting units to GeV
xe137_df['GeV'] = xe137_df.Xemunrg * 1e-3

# Make a histogram of the data
xe137_count, _ = np.histogram(xe137_df.GeV.values, bins = bins)

# Divide the muon energy by the random flat distribution
xe137_exp      = xe137_count / binned_sim_muons

xe137_exp_err  = xe137_exp * np.sqrt(1 / xe137_count + 1 / numberofmuons)
print(xe137_exp_err)

In [ ]:
plt.figure(figsize=(7,7))


plt.xlabel('Muon Energy (GeV)',fontsize=20)
plt.ylabel(' $^{137}$Xe per Muon per '+str(stp_size)+ ' GeV (10$^{-4}$)',fontsize=20)
plt.yticks(fontsize=18) 
plt.xticks(fontsize=18)

plt.tight_layout()
#plt.semilogy()
plt.ylim(0,8.5)
#plt.bar(bin_centers,(binned_Xe137data[0]/binned_sim_muons),width=stp_size,color="deepskyblue",label="0% Helium")    
plt.errorbar(bin_centers, (binned_Xe137data[0]/binned_sim_muons*10**4 ), fmt='^',
                 xerr = np.diff(bins) / 2, yerr = xe137_exp_err*10**4,color="deepskyblue",label="0% $^{3}$He",ms=5) 
             
             
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.legend(fontsize=18, loc="upper left", title="Mass Fraction",title_fontsize=18)
plt.savefig("xepermuenerg.png",dpi=250,bbox_inches='tight')
#plt.savefig("xepermuenerg.pdf")
plt.show()